# 1 Seq2Seq

https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # batch
epochs = 20  # epoch
latent_dim = 256  # context vactor 차원수
num_samples = 10000  # 학습데이터 검수
data_path = 'fra.txt' # 학습데이터 파일

# 학습데이터 벡터화
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t') #데이터는 tab 으로 구분
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters)) #입력 시퀀스 문자
target_characters = sorted(list(target_characters)) #출력 시퀀스 문자
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts]) #입력 시퀀스의 최대 길이
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # 출력 시퀀스의 최대 길이

In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)]) #문자 : 인덱스 dictionary
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)]) #문자 : 인덱스 dictionary

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32') 
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1. #padding
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

# encoder 정의
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c] #encoder_outputs 는 사용하지 않음

In [ ]:
# encoder_states를 최초 state로 사용
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Seq2Seq 학습
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

model.save('s2s.h5')

Epoch 1/20
125/125 [==============================] - 51s 412ms/step - loss: 1.1797 - accuracy: 0.7250 - val_loss: 1.0387 - val_accuracy: 0.7087
Epoch 2/20
125/125 [==============================] - 47s 378ms/step - loss: 0.8596 - accuracy: 0.7687 - val_loss: 0.8392 - val_accuracy: 0.7683
Epoch 3/20
125/125 [==============================] - 48s 380ms/step - loss: 0.6901 - accuracy: 0.8055 - val_loss: 0.7163 - val_accuracy: 0.7894
Epoch 4/20
125/125 [==============================] - 48s 386ms/step - loss: 0.5952 - accuracy: 0.8267 - val_loss: 0.6481 - val_accuracy: 0.8101
Epoch 5/20
125/125 [==============================] - 48s 385ms/step - loss: 0.5429 - accuracy: 0.8410 - val_loss: 0.6134 - val_accuracy: 0.8178
Epoch 6/20
125/125 [==============================] - 48s 384ms/step - loss: 0.5056 - accuracy: 0.8515 - val_loss: 0.5764 - val_accuracy: 0.8297
Epoch 7/20
 43/125 [=========>....................] - ETA: 28s - loss: 0.4805 - accuracy: 0.8581

In [ ]:
# 예측 모델링
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items()) # 인덱스 : 문자 dictionary
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items()) # 인덱스 : 문자 dictionary

In [ ]:
#예측 수행

def decode_sequence(input_seq):
    # 입력 시퀀스를 컨텍스트 벡터로 변환
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # SOS(Start of Sequence) 설정
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 한글자 예측하고 결과를 다시 디코더의 입력으로 하여 예측을 반복
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 토큰 추출
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # EOS (End of Seqence). 예제에서는 개행문자 등장시 혹은 최대길이를 달성하면 출력을 멈춤
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 다음 디코더 입력에 사용할 벡터 생성
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 스테이트 갱신
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)